In [1]:
# ! pip install pandas 
# ! pip install fuzzywuzzy

In [7]:
import pandas as pd
import re
from fuzzywuzzy import fuzz

In [2]:
dairy = pd.read_csv('raw_data/zehrs/dairy_and_eggs.csv')

In [3]:
dairy.head()

,category,brand,product,product_name,price,per_unit_price
0,dairy_and_eggs,No Name,Salted Butter,No NameSalted Butter454 g,$6.19ea,$1.36/ 100g
1,dairy_and_eggs,Neilson,Half and Half Cream,NeilsonHalf and Half Cream1 l,$4.79ea,$0.48/ 100ml
2,dairy_and_eggs,Neilson,Partly Skimmed Milk 1% MF,NeilsonPartly Skimmed Milk 1% MF4 l,$5.69ea,$0.14/ 100ml
3,dairy_and_eggs,No Name,Unsalted Butter,No NameUnsalted Butter454 g,$6.19ea,$1.36/ 100g
4,dairy_and_eggs,Neilson,Homogenized Milk 3.25%,NeilsonHomogenized Milk 3.25%4 l,$6.69ea,$0.17/ 100ml


In [4]:
# lets say a users search term is 
search_term = '2% Milk'

# and we have a product 
product = 'Neilson Chocolate Partly Skimmed Milk 2% MF4 l'
# product= 'Gray RidgePremium Large Eggs18 eggs'

# similarity: 
sim = fuzz.token_sort_ratio(product.lower(), search_term.lower())
sim

24

^ kinda low 

In [225]:
# compared to if the search term is 
search_term = 'Milk 2%'

# and we have a product 
product = 'NeilsonPartly Skimmed Milk 2% MF4 l'

# similarity: 
sim = fuzz.partial_ratio(product.lower(), search_term.lower())
sim

100

^ this is good, we want this as high as possible. but people probably wouldnt default to this, they would do the one before 

In [226]:
# now what if we break up search terms into each word
search_term = '2% Milk'
product = 'NeilsonPartly Skimmed Milk 2% MF4 l'

total_ratio = 0
for term in search_term.split():
    sim = fuzz.partial_ratio(product.lower(), term.lower())
    total_ratio += sim

total_ratio / len(search_term.split()) # essentially the avg ratio value 

100.0

### Load all data into one df

In [270]:
dairy = pd.read_csv('raw_data/zehrs/dairy_and_eggs.csv')
bakery = pd.read_csv('raw_data/zehrs/bakery.csv')
meat = pd.read_csv('raw_data/zehrs/meat.csv')
produce = pd.read_csv('raw_data/zehrs/produce.csv')
seafood = pd.read_csv('raw_data/zehrs/seafood.csv')

df = pd.DataFrame()
df = df.append(dairy)
df = df.append(bakery)
df = df.append(meat)
df = df.append(produce)
df = df.append(seafood)

df['store'] = 'zehrs'


df.head()

,category,brand,product,product_name,price,per_unit_price,store
0,dairy_and_eggs,No Name,Salted Butter,No NameSalted Butter454 g,$6.19ea,$1.36/ 100g,zehrs
1,dairy_and_eggs,Neilson,Half and Half Cream,NeilsonHalf and Half Cream1 l,$4.79ea,$0.48/ 100ml,zehrs
2,dairy_and_eggs,Neilson,Partly Skimmed Milk 1% MF,NeilsonPartly Skimmed Milk 1% MF4 l,$5.69ea,$0.14/ 100ml,zehrs
3,dairy_and_eggs,No Name,Unsalted Butter,No NameUnsalted Butter454 g,$6.19ea,$1.36/ 100g,zehrs
4,dairy_and_eggs,Neilson,Homogenized Milk 3.25%,NeilsonHomogenized Milk 3.25%4 l,$6.69ea,$0.17/ 100ml,zehrs


## Data Cleaning

In [79]:
re.sub("(/[0-9] +.*$)", '', '12.10/1 kg')

'12.10'

In [ ]:
# for volume - convert to ml 
# for wt - convert to g 

In [236]:
units = []
for index, row in df.iterrows():
    per_unit_price = row.per_unit_price
    try: 
        unit = per_unit_price.split('/')[1]
        if unit not in units: 
            units.append(unit)
    except: continue # print(per_unit_price)

units[:10]

[' 100g', ' 100ml', ' 1ea', ' 1kg']

In [256]:
'1.25/100g'.split('/')[0].replace('$', '')

'1.25'

In [260]:
float('1kg'.split('kg')[0])*1000

1000.0

In [263]:
per_unit_prices = []
unit_types = []
for index, row in df.iterrows():
    per_unit_price = row.per_unit_price
    if not isinstance(per_unit_price, float):  # not nan
        unit = per_unit_price.split('/')[1]
        price = float(per_unit_price.split('/')[0].replace('$', '').replace(',', ''))
        unit_type = ''
        if 'kg' in unit: 
            gram = float(unit.split('kg')[0])*1000 # 1000 g in kg
            pup = price / gram
            unit_type = 'g'
        elif 'g' in unit:
            gram = float(unit.split('g')[0])
            pup = price / gram
            unit_type = 'g'
        elif 'ml' in unit:
            ml = float(unit.split('ml')[0])
            pup = price / ml
            unit_type = 'ml'
    else: 
        pup = row.price
        unit_type = ''
    
    per_unit_prices.append(pup)
    unit_types.append(unit_type)
    
df['per_unit_price2'] = per_unit_prices
df['unit_type'] = unit_types
df.head()

,category,brand,product,product_name,price,per_unit_price,store,per_unit_price2,unit_type
0,dairy_and_eggs,No Name,Salted Butter,No NameSalted Butter454 g,$6.19ea,$1.36/ 100g,zehrs,0.0136,g
1,dairy_and_eggs,Neilson,Half and Half Cream,NeilsonHalf and Half Cream1 l,$4.79ea,$0.48/ 100ml,zehrs,0.0048,ml
2,dairy_and_eggs,Neilson,Partly Skimmed Milk 1% MF,NeilsonPartly Skimmed Milk 1% MF4 l,$5.69ea,$0.14/ 100ml,zehrs,0.0014,ml
3,dairy_and_eggs,No Name,Unsalted Butter,No NameUnsalted Butter454 g,$6.19ea,$1.36/ 100g,zehrs,0.0136,g
4,dairy_and_eggs,Neilson,Homogenized Milk 3.25%,NeilsonHomogenized Milk 3.25%4 l,$6.69ea,$0.17/ 100ml,zehrs,0.0017,ml


In [272]:
# loblaw cleaning 

import re 

prices = []
per_unit_prices = []
unit_types = []

for index, row in df.iterrows():
    s = row.price
    s = re.findall('(?:[\£\$\€]{1}[,\d]+.?\d*)',s)[-1]
    s = s.replace('$', '') # remove $
    
    per_unit_price = row.per_unit_price
    if not isinstance(per_unit_price, float):  # not nan
        unit = per_unit_price.split('/')[1]
        price = float(per_unit_price.split('/')[0].replace('$', '').replace(',', ''))
        unit_type = ''
        if 'kg' in unit: 
            gram = float(unit.split('kg')[0])*1000 # 1000 g in kg
            pup = price / gram
            unit_type = 'g'
        elif 'g' in unit:
            gram = float(unit.split('g')[0])
            pup = price / gram
            unit_type = 'g'
        elif 'ml' in unit:
            ml = float(unit.split('ml')[0])
            pup = price / ml
            unit_type = 'ml'
    else: 
        pup = row.price
        unit_type = ''
    
    prices.append(s)
    per_unit_prices.append(pup)
    unit_types.append(unit_type)
    
df['per_unit_price2'] = per_unit_prices
df['unit_type'] = unit_types
df['price2'] = prices
    


In [273]:
df.head()

,category,brand,product,product_name,price,per_unit_price,store,per_unit_price2,unit_type,price2
0,dairy_and_eggs,No Name,Salted Butter,No NameSalted Butter454 g,$6.19ea,$1.36/ 100g,zehrs,0.0136,g,6.19
1,dairy_and_eggs,Neilson,Half and Half Cream,NeilsonHalf and Half Cream1 l,$4.79ea,$0.48/ 100ml,zehrs,0.0048,ml,4.79
2,dairy_and_eggs,Neilson,Partly Skimmed Milk 1% MF,NeilsonPartly Skimmed Milk 1% MF4 l,$5.69ea,$0.14/ 100ml,zehrs,0.0014,ml,5.69
3,dairy_and_eggs,No Name,Unsalted Butter,No NameUnsalted Butter454 g,$6.19ea,$1.36/ 100g,zehrs,0.0136,g,6.19
4,dairy_and_eggs,Neilson,Homogenized Milk 3.25%,NeilsonHomogenized Milk 3.25%4 l,$6.69ea,$0.17/ 100ml,zehrs,0.0017,ml,6.69


# flipp data

In [5]:
freshco = pd.read_csv('raw_data/freshco/flyer_deals.csv')
freshco['store'] = 'freshco'

walmart = pd.read_csv('raw_data/walmart/flyer_deals.csv')
walmart['store'] = 'walmart'

sobeys = pd.read_csv('raw_data/sobeys/flyer_deals.csv')
sobeys['store'] = 'sobeys'

foodbasics = pd.read_csv('raw_data/food_basics/flyer_deals.csv')
foodbasics['store'] = 'foodbasics'

zehrs = pd.read_csv('raw_data/zehrs/flyer_deals.csv')
zehrs['store'] = 'zehrs'

valu_mart = pd.read_csv('raw_data/valu_mart/flyer_deals.csv')
valu_mart['store'] = 'valu_mart'

no_frills = pd.read_csv('raw_data/no_frills/flyer_deals.csv')
no_frills['store'] = 'no_frills'

df = pd.DataFrame()
df = df.append(freshco)
df = df.append(walmart)
df = df.append(sobeys)
df = df.append(foodbasics)
df = df.append(valu_mart)
df = df.append(no_frills)

df.head()

,product_name,price,store
0,Compliments Bathroom Tissue 12=24 rolls,$4.99 with Scene+ Card,freshco
1,Philadelphia Cream Cheese 227-250 g,$3.49 with Scene+ Card,freshco
2,Compliments Frozen Fruit 600 g,$3.99 with Scene+ Card,freshco
3,Best Buy Frozen Vegetables 2 kg,$4.49 with Scene+ Card,freshco
4,RS Premium Natural Almonds 300 g Product of USA,$2.99 with Scene+ Card,freshco


In [10]:
re.findall('(?:[/lbkgml]+)','2.99 lb')[-1]

'lb'

In [12]:
'/$' in '$2.99 /lb'

False

In [17]:
# flipp cleaning 

prices = []
drop_items = []
per_unit_prices = []
unit_types = []
for index, row in df.iterrows():
    s = row.price
    store = row.store
    
    price = ''
    unit = ''
    
    # remove items that aren't groceries or for general pop 
    if '*' in s: drop = 1 # electronics 
    elif 'Scene+' in s: drop = 1 # offer not avail to everyone
    elif 'when you' in s: drop = 1 # offer not avail to everyone
    else: drop = 0
        
    # multiple units for deal  
    if '/$' in s: 
        num = int(s.split('/')[0])
        price_txt = re.findall('(?:[\£\$\€]{1}[,\d]+.?\d*)',s)[-1]
        price = float(price_txt.replace('$', ''))
        price = price/num
        price = round(price,2)
    
    try:
        price_txt = re.findall('(?:[\£\$\€]{1}[,\d]+.?\d*)',s)[-1] # take last price if multiple prices listed since it is the single unit price 
        price = price_txt.replace('$', '')
    except: 
        price = None
        drop = 1
    
    
    # see if there are units 
    
    # in price column 
    try: 
        unit = re.findall('(?:[/lbkgml]+)',s)[-1] # this may error out in which case no units 
        
        if unit == '/': unit = None

        if unit is not None:  # not nan
            pup,unit_type =  find_per_unit_price(price, unit)
        else: 
            pup = price
            unit_type = None
    except: 
        pup = price
        unit_type = None
    
    # in product description 
    if unit_type == None: # use above price per_unit parsing  as default / standard 
        try: 
            if store == 'zehrs' or store == 'no_frills' or store == 'valu_mart' and unit :
                desc = row.product_name
                if ',' in desc: unit = decs.split(',')[-1]
                else: unit = None
                unit = re.findall('(([0-9]+) g|([0-9]+) kg|([0-9]+)\'s)|[0-9]+ pk',unit)[-1][0]

            elif store == 'freshco':
                desc = row.product_name
                unit = re.findall('(([0-9]+) g|([0-9]+) kg|([0-9]+)\'s)|[0-9]+ pk',desc)[-1][0]

            else:
                unit = None
                
            if unit is not None:  # not nan
                pup,unit_type =  find_per_unit_price(price, unit)
            else: 
                pup = price
                unit_type = None
        except: 
            pup = price
            unit_type = None
    
    prices.append(price)
    per_unit_prices.append(pup)
    unit_types.append(unit_type)    
    drop_items.append(drop)
    
        
df['price2'] = prices
df['per_unit_price2'] = per_unit_prices
df['unit_type'] = unit_types

df['drop_col'] = drop_items 
df = df[df['drop_col'] == 0]
df = df.drop(columns=['drop_col'])    


In [48]:
def find_per_unit_price(price, unit):
    unit_type = ''
    if 'kg' in unit: 
        gram = unit.split('kg')[0]
        if gram == '/': gram = 1
        pup = float(price) / (float(gram)*1000) # 1000 g in kg
        unit_type = 'g'
    elif 'g' in unit:
        gram = unit.split('g')[0]
        if gram == '/': gram = 1
        pup = float(price) / float(gram)
        unit_type = 'g'
    elif 'lb' in unit:
        gram = unit.split('lb')[0]
        if gram == '/' or gram == '': gram = 1
#         if gram == '': gram = price.split('lb')[0]
        pup = float(price) / (float(gram)*453.592)
        unit_type = 'g'
    elif 'ml' in unit:
        ml = unit.split('ml')[0]
        if ml == '/': ml = 1
        pup = float(price) / float(ml)
        unit_type = 'ml'
    elif 'l' in unit:
        ml = unit.split('l')[0]
        if unit == '/': ml = 1
        pup = float(price) / (float(ml)*1000)
        unit_type = 'ml'
    elif "'s'" in unit:
        num = unit.split("'s'")[0]
        pup = float(price) / (float(num))
        unit_type = 'number_of_units'
    elif 'pk' in unit:
        num = unit.split("pk")[0]
        pup = float(price) / (float(num))
        unit_type = 'number_of_units'
    elif 'pack' in unit:
        num = unit.split("pack")[0]
        pup = float(price) / (float(num))
        unit_type = 'number_of_units'
    else: 
        pup = price
        unit_type = None
            
    return pup, unit_type

In [46]:
'299'.split(' ')[0]

'299'

In [49]:
s = '$2.99 lb'
store = row.store

price = ''
unit = ''

# remove items that aren't groceries or for general pop 
if '*' in s: drop = 1 # electronics 
elif 'Scene+' in s: drop = 1 # offer not avail to everyone
elif 'when you' in s: drop = 1 # offer not avail to everyone
else: drop = 0

# multiple units for deal  
if '/$' in s: 
    num = int(s.split('/')[0])
    price_txt = re.findall('(?:[\£\$\€]{1}[,\d]+.?\d*)',s)[-1]
    price = float(price_txt.replace('$', ''))
    price = price/num
    price = round(price,2)

try:
    price_txt = re.findall('(?:[\£\$\€]{1}[,\d]+.?\d*)',s)[-1] # take last price if multiple prices listed since it is the single unit price 
    price = price_txt.replace('$', '')
except: 
    price = None
    drop = 1

print(price)
# see if there are units 

# in price column 
# try: 
unit = re.findall('(?:[/lbkgml]+)',s)[-1] # this may error out in which case no units 

if unit == '/': unit = None

if unit is not None:  # not nan
    pup,unit_type =  find_per_unit_price(price, unit)
else: 
    pup = price
    unit_type = None
# except: 
#     pup = price
#     unit_type = None

        
pup

2.99


0.006591827016349495

In [29]:
# if store == 'zehrs' or store == 'no_frills' or store == 'valu_mart' and unit :
#     desc = '2.99 lb'
#     if ',' in desc: unit = decs.split(',')[-1]
#     else: unit = None
#     unit = re.findall('(([0-9]+) g|([0-9]+) kg|([0-9]+)\'s)|[0-9]+ pk',unit)[-1][0]
    
# unit

unit = re.findall('(?:[/lbkgml]+)','2.99 lb')[-1] # this may error out in which case no units 

print(unit)
if unit == '/': unit = None

if unit is not None:  # not nan
    
    pup,unit_type =  find_per_unit_price('2.99 lb', unit)
else: 
    pup = price
    unit_type = None
  
    
# gram = '2.99 lb'.split('lb')[0]
# float(gram)


lb


ValueError: could not convert string to float: '2.99 lb'

In [346]:
if store == 'zehrs' or store == 'no_frills' or store == 'valu_mart' and unit :
    desc = 'PORK BACK RIBS, 2'
    if ',' in desc: unit = desc.split(',')[-1]
    else: unit = None
    unit = re.findall('(([0-9]+) g|([0-9]+) kg|([0-9]+) lb|([0-9]+)\'s)|[0-9]+ pk',unit)[-1][0]
        
unit

'350 g'

In [311]:
# zehrs - no frills - valu mart, split on the column (last item in list ) find unit or 's

# fresh co, find pattern like ### g or ###/kg , ## pk

# sobeys, has in price value so no need for parsing 


## Search

In [26]:
search_term= '2% milk'

max_sims = [0,0,0]
item_name = ['', '', '']

similarities = {}
prices = {}

for index, row in df.iterrows():
    product_text = row['product']
    price = row.price
    total_ratio = 0
    similarity = 0
    for term in search_term.split():
        sim = fuzz.partial_ratio(product_text.lower(), term.lower())
        total_ratio += sim
    
    similarity = total_ratio / len(search_term.split())
       
    if similarity > 80: 
        similarities[product_text] = similarity
        prices[product_text] = price

        
print(similarities)

{'Organic 2% Milk': 100.0, 'Trutaste Lactose Free 2% Milk': 100.0, 'Chocolate Ultrafiltered Partly Skimmed Milk Lactose Free 2% M.F.': 100.0, 'Chocolate Lactose Free 2% Partly Skimmed Ultrafiltered Milk': 100.0, '2% Lactose Free Chocolate Enriched Milk': 100.0, 'TruTaste 2% Milk': 100.0, 'Organic Partly Skimmed 2% Milk': 100.0, '2% Milk': 100.0, 'Goat Milk 2%MF': 100.0, 'Ultrafiltered Partly Skimmed Milk Lactose Free 2% M.F': 100.0, 'TruTaste Lactose Free 2% Milk': 100.0, 'True Taste, 2% Milk': 100.0, '2% M.F. Partly Skimmed Milk': 100.0}


In [33]:
len(options)

16

^ I think thats a good solution 

## Scoping some things out: 

Say someone searches 2% milk - we then have to retrieve it from the data 

A few things I think we have to do:

1. Categorize the search term to match our data categories (eg: bakery, dairy_and_eggs, meat, etc) 
2. Search that dataset as above to find top ranking products for each store and flyer data set 

I think the best way to achieve 1. is with a classifier 

pretty much we would need to extract keywords from all our products (or split product names into seprate strings and each string would then be matched to a category) 

Train a classifier on that data ^ 

Input a search term and then output a category pred and then search that dataset (would need the prediction to be pretty high to be confident) 

Another idea - put all data into one dataset/csv and then we can just loop through every product - honestly given the dataset size and how fast we can similarity match, I think that is doable from a runtime perspective even though it seems like a lazy idea (it'd be a good start for a MVP) 

- we can start with this and if we find its too slow then we can go from there on a more complex solution, this is a good jumping off point 

I still need to clean the product names (hopefully just by cleaning up the scrapers) 

Can start with fuzzy wuzzy but look into a python package search engine for comparison in performance 